In [0]:
hosa_patients_path = "/mnt/rcm/bronze/hosa/patients"
hosb_patients_path = "/mnt/rcm/bronze/hosb/patients"

print(f"Reading Hospital A patients from: {hosa_patients_path}")
print(f"Reading Hospital B patients from: {hosb_patients_path}")

# Reading Hospital A patient data and creating a temporary view
df_hosa = spark.read.parquet(hosa_patients_path)
df_hosa.createOrReplaceTempView("patients_hosa_bronze") # Renamed for clarity

# Reading Hospital B patient data and creating a temporary view
df_hosb = spark.read.parquet(hosb_patients_path)
df_hosb.createOrReplaceTempView("patients_hosb_bronze") # Renamed for clarity

print("Bronze layer patient data loaded into temporary views: patients_hosa_bronze, patients_hosb_bronze")

In [0]:
%sql
SELECT * FROM patients_hosa_bronze LIMIT 10

In [0]:
%sql
SELECT * FROM patients_hosb_bronze LIMIT 10

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW cdm_patients AS
SELECT CONCAT(SRC_PatientID, '-', datasource) AS Patient_Key, *
FROM (
    -- Data from Hospital A (patients_hosa_bronze)
    SELECT
        PatientID AS SRC_PatientID,
        FirstName,
        LastName,
        MiddleName,
        SSN,
        PhoneNumber,
        Gender,
        DOB,
        Address,
        ModifiedDate,
        'hosa' AS datasource -- Explicitly setting datasource for Hospital A
    FROM patients_hosa_bronze

    UNION ALL

    -- Data from Hospital B (patients_hosb_bronze)
    SELECT
        ID AS SRC_PatientID,
        F_Name AS FirstName,
        L_Name AS LastName,
        M_Name AS MiddleName,
        SSN,
        PhoneNumber,
        Gender,
        DOB,
        Address,
        Updated_Date AS ModifiedDate,
        'hosb' AS datasource -- Explicitly setting datasource for Hospital B
    FROM patients_hosb_bronze
);

In [0]:
%sql
SELECT * FROM cdm_patients LIMIT 20

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW quality_checks AS
SELECT
    Patient_Key,
    SRC_PatientID,
    FirstName,
    LastName,
    MiddleName,
    SSN,
    PhoneNumber,
    Gender,
    DOB,
    Address,
    ModifiedDate AS SRC_ModifiedDate, -- Renamed for clarity to indicate source modification date
    datasource,
    CASE
        WHEN SRC_PatientID IS NULL OR dob IS NULL OR firstname IS NULL OR lower(firstname) = 'null' THEN TRUE
        ELSE FALSE
    END AS is_quarantined
FROM cdm_patients;


In [0]:
%sql
SELECT * FROM quality_checks
ORDER BY is_quarantined DESC
LIMIT 20

In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver.patients (
    Patient_Key STRING,        -- Unique identifier for a patient across all sources
    SRC_PatientID STRING,      -- Original patient ID from the source system
    FirstName STRING,
    LastName STRING,
    MiddleName STRING,
    SSN STRING,
    PhoneNumber STRING,
    Gender STRING,
    DOB DATE,
    Address STRING,
    SRC_ModifiedDate TIMESTAMP, -- Last modification date from the source system
    datasource STRING,          -- Source system identifier (e.g., 'hosa', 'hosb')
    is_quarantined BOOLEAN,     -- Flag from data quality checks
    inserted_date TIMESTAMP,    -- Timestamp when this specific record version was inserted into Silver
    modified_date TIMESTAMP,    -- Timestamp when this specific record version was last modified/deactivated
    is_current BOOLEAN          -- Flag indicating if this is the currently active version (TRUE) or historical (FALSE)
)
USING DELTA;

In [0]:
%sql
DESCRIBE DETAIL silver.patients;

In [0]:
%sql
MERGE INTO silver.patients AS tgt
USING quality_checks AS src
ON tgt.Patient_Key = src.Patient_Key AND tgt.is_current = true
WHEN MATCHED
AND (
    -- Check for any column changes (excluding inserted_date, modified_date, is_current)
    -- If any of these columns are different, it means the record has been updated in the source.
    tgt.SRC_PatientID      <> src.SRC_PatientID OR
    tgt.FirstName          <> src.FirstName OR
    tgt.LastName           <> src.LastName OR
    tgt.MiddleName         <> src.MiddleName OR
    tgt.SSN                <> src.SSN OR
    tgt.PhoneNumber        <> src.PhoneNumber OR
    tgt.Gender             <> src.Gender OR
    tgt.DOB                <> src.DOB OR
    tgt.Address            <> src.Address OR
    tgt.SRC_ModifiedDate   <> src.SRC_ModifiedDate OR
    tgt.datasource         <> src.datasource OR
    tgt.is_quarantined     <> src.is_quarantined
)
THEN UPDATE SET
    tgt.is_current = false,       -- Mark the old version as no longer current
    tgt.modified_date = current_timestamp() -- Record the time it became historical
;


In [0]:
%sql
MERGE INTO silver.patients AS tgt
USING quality_checks AS src
ON tgt.Patient_Key = src.Patient_Key AND tgt.is_current = true
WHEN NOT MATCHED
THEN INSERT (
    Patient_Key,
    SRC_PatientID,
    FirstName,
    LastName,
    MiddleName,
    SSN,
    PhoneNumber,
    Gender,
    DOB,
    Address,
    SRC_ModifiedDate,
    datasource,
    is_quarantined,
    inserted_date,
    modified_date,
    is_current
)
VALUES (
    src.Patient_Key,
    src.SRC_PatientID,
    src.FirstName,
    src.LastName,
    src.MiddleName,
    src.SSN,
    src.PhoneNumber,
    src.Gender,
    src.DOB,
    src.Address,
    src.SRC_ModifiedDate,
    src.datasource,
    src.is_quarantined,
    current_timestamp(), -- Set inserted_date to current timestamp for this version
    current_timestamp(), -- Set modified_date to current timestamp for this version
    true                 -- Mark this new record/version as current
);


In [0]:
%sql
SELECT
    COUNT(*) AS version_count,
    Patient_Key
FROM silver.patients
GROUP BY Patient_Key
ORDER BY version_count DESC;